(model-building)=

# Model Building

The modeling framework of ``ELISA`` is designed to be flexible, allowing you
to construct arbitrarily complex models by combining models from ELISA or
Xspec, as well as your custom models. Links between parameters
across different model components can also be established. These models can
then be fitted to the spectral datasets.

## The Model Interface

In the context of X/$\gamma$-ray spectral fitting, a spectral model is a
photon flux model. The model may be composed by a single flux component, or
many additive flux components, and these additive components can be further
modified by multiplicative or convolution components. You can check
``ELISA``'s built-in model components from the {ref}`api`:

- [additive](#elisa.models.add)
- [multiplicative](#elisa.models.mul)
- [convolution](#elisa.models.conv)

### Create a Model
To use the spectral models in ``ELISA``, you can run the following code to import
all model classes of three types from the ``elisa.models`` module:

In [ ]:
from elisa.models import *

After the import, you can create an instance of a model by calling the model
class. For example, the following code creates a [`PowerLaw`](#elisa.models.add.PowerLaw)
photon flux model:

In [ ]:
m = PowerLaw()
m

The string representation of the model shows its name along with its model type,
the components and corresponding parameters. We can see that the parameters are
initialized with default values, bounds, and the prior distribution.
We will discuss how to configure the default values, bounds, and priors of
these parameters in the [parameter interface](#parameter-interface) section.

Next, we can create a new model by modifying the power-law model with a photoelectric absorption component [`PhAbs`](#elisa.models.mul.PhAbs). We first create the absorption model along with the `angr` abundance table and the `vern` photoelectric cross-section.

In [ ]:
phabs = PhAbs(abund='angr', xsect='vern')
phabs

Note that [`PhAbs`](#elisa.models.mul.PhAbs) uses `angr` abundance and `vern`
cross-section by default, and thus ``PhAbs(abund='angr', xsect='vern')`` is
equivalent to ``PhAbs()``.

Then, we can create a new model by multiplying the power-law model with the
photoelectric absorption component:

In [ ]:
m2 = phabs * PowerLaw()
m2

Assume the model is for an extragalactic source with a redshift measurement
like $z=1.5$, we convolve the photon flux model with [`ZAShift`](#elisa.models.conv.ZAShift)
to account for the redshift. We first create the redshift component,

In [ ]:
redshift = ZAShift(z=1.5)
redshift

We can see that the redshift parameter is fixed to 1.5. Now we can create a new
model by convolving the previous model with the redshift component:

In [ ]:
m3 = redshift(m2)
m3

The model is now a power-law model with photoelectric absorption, and further
modified by the redshift.

Note that the same model can be created with a single line code:

In [ ]:
m4 = ZAShift(z=1.5)(PhAbs() * PowerLaw())
m4

Now let's have a look at the spectral shape of the model we just built,

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['axes.formatter.min_exponent'] = 3

# Compile the model and get CompiledModel instance
compiled_model = m4.compile()

# Create a photon energy grid used to evaluate the model
photon_egrid = np.linspace(0.3, 15.0, 1000)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])

# Evaluate the model with the default parameters
# photon flux N(E) [s^-1 cm^-2 keV^-1]
ne = compiled_model.ne(photon_egrid)

# You can also evaluate the model with custom
# parameters, by specifying either a full set
# or a subset of parameters to CompiledModel.ne
custom_params = {'PowerLaw.alpha': 2.0}
ne2 = compiled_model.ne(photon_egrid, params=custom_params)

ne3 = compiled_model.ne(photon_egrid, params={'ZAShift.z': 0.0})

# For Fv or vFv plot, we can use ene and eene
# ene = compiled_model.ene(photon_egrid)
# eene = compiled_model.eene(photon_egrid)

plt.step(egrid_mid, ne, label='default')
plt.step(egrid_mid, ne2, label='alpha=2.0')
plt.step(egrid_mid, ne3, label='z=0.0')
plt.legend()
plt.title(m4.name)
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

Great! We have successfully built a model and inspected its spectral shape.
You can build any model you want in a similar way.

```{admonition} Tip
:class: tip

The model's $F_\nu$ and $\nu F_\nu$ values can be accessed by [`CompiledModel`](#elisa.models.model.CompiledModel)'s [`ene`](#elisa.models.model.CompiledModel.ene) and
[`eene`](#elisa.models.model.CompiledModel.eene) methods.

In addition, [`CompiledModel`](#elisa.models.model.CompiledModel) provides tools
to calculate [`flux`](#elisa.models.model.CompiledModel.flux), isotropic-equivalent
[`luminosity`](#elisa.models.model.CompiledModel.lumin) and [`energy`](#elisa.models.model.CompiledModel.eiso).
```

### Use XSPEC Models

``ELISA`` can make use of ``XSPEC`` [models](https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/Models.html). Before using them, you need to follow the {ref}`installation` to install the [``xspex``](https://github.com/wcxve/xspex) package, which provides ``JAX`` interface for ``XSPEC`` models.

Once [``xspex``](https://github.com/wcxve/xspex) package is installed successfully, import ``ELISA``'s wrapper of ``XSPEC`` models by

In [ ]:
from elisa.models import xs

We can build a model combining models from both ``ELISA`` and ``XSPEC``.
For examle,

In [ ]:
m5 = TBAbs() * xs.apec()
m5

In [ ]:
photon_egrid = np.linspace(0.3, 15.0, 2000)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
ne = m5.compile().ne(photon_egrid)
plt.step(egrid_mid, ne)
plt.title(m5.name)
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

In [ ]:
m6 = xs.thcomp()(PowerLaw())
m6

In [ ]:
photon_egrid = np.linspace(60.0, 90.0, 1000)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
ne = m6.compile().ne(photon_egrid)
plt.step(egrid_mid, ne)
plt.title(m6.name)
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

### Use Custom Models

You can also include custom models when building models.
See {ref}`custom-model` tutorial for details on how to make a custom model.

### Conclusion

By leveraging the model interface of ``ELISA``, you can construct arbitrarily
complex spectral models to suit your needs.

In the next section, we will discuss the parameter configuration for the model.

(parameter-interface)=
## The Parameter Interface

``ELISA`` provide an intuitive way to configure the parameters of the model
components.

As we have seen in the previous section, the model components have parameters
associated with them. These parameters have default values, bounds, and prior
distributions defined. Indeed, these parameters are instances of the
[`Parameter`](#elisa.models.parameter.Parameter) class.

### The Uniform Parameter

When creating a model without specifying the configuration of parameters,
[`UniformParameter`](#elisa.models.parameter.UniformParameter) instances are
automatically created and assigned to model components.

For example, when we initialize a [`PowerLaw`](#elisa.models.add.PowerLaw)
simply by ``PowerLaw()``, information is read from the `PowerLaw._config`
attribute and passed to the [`UniformParameter`](#elisa.models.parameter.UniformParameter)
to create the parameter objects for the power-law model. The information includes
the default values, bounds, and flags to indicate whether the parameters are
parameterized in logarithmic space and whether to be fixed. This is also the
case for other model components.

In [ ]:
for i in PowerLaw._config:
    print(i, end='\n\n')

In [ ]:
PowerLaw()

We can see that the configuration of parameters matches the information printed
above.

Usually, it is fine to use the default configuration. However, you can customize
the parameters for your need. For example, you can create [`UniformParameter`](#elisa.models.parameter.Parameter) instances with custom default values and bounds, and then
passed to the model constructor:

In [ ]:
from elisa import UniformParameter

alpha = UniformParameter(
    name='alpha', default=2.0, min=0.0, max=5.0, log=False, fixed=False
)

K = UniformParameter(
    name='K', default=10, min=1e-5, max=1e5, log=True, fixed=False
)

PowerLaw(alpha=alpha, K=K)

``ELISA`` also provides several convenient ways to set up the [`UniformParameter`](#elisa.models.parameter.UniformParameter) for model components, without the need to create the [`UniformParameter`](#elisa.models.parameter.UniformParameter) instances explicitly:

Passing size one float sequence to the model constructor will create a [`UniformParameter`](#elisa.models.parameter.UniformParameter) with the float as the default value,

In [ ]:
PowerLaw(alpha=[1.7])

Passing three-sequence will create a [`UniformParameter`](#elisa.models.parameter.UniformParameter) with the first element as the default value, the second and third elements as the minimum and maximum values,

In [ ]:
Blackbody(kT=(10, 2, 30), K=[1.1, 1e-5, 1e4])

Passing a four-sequence will create a [`UniformParameter`](#elisa.models.parameter.UniformParameter) with the first three elements as the default, minimum and maximum values, and the fourth element as the flag to indicate whether the parameter is logarithmically parameterized,

In [ ]:
TBAbs(nH=[3.0, 0.01, 10.0, True])

And finally, passing a float will create a UniformParameter with the float as the default value, and the parameter is fixed to this value,

In [ ]:
ZAShift(z=4.2)

### Other Parameters

In addition to the [`UniformParameter`](#elisa.models.parameter.UniformParameter), ``ELISA`` provides three other types of parameters.

#### DistParameter

You can create a [`DistParameter`](#elisa.models.parameter.DistParameter) instance
by passing a ``NumPyro``'s [probability distribution](https://num.pyro.ai/en/stable/distributions.html)
instance. This is useful when you want to use non-uniform priors for parameters
in Bayesian analysis. For example, we can create a multiplicative [``Constant``](#elisa.models.mul.Constant) component with a Gaussian prior in $\mathbb{R^+}$.

In [ ]:
import numpyro.distributions as dist

from elisa import DistParameter

f = DistParameter(
    name='f',
    dist=dist.TruncatedNormal(
        loc=1.0,
        scale=0.2,
        low=0.0,
    ),
    default=1.0,
)
Constant(f=f)

#### ConstantInterval

When assigning [`ConstantInterval`](#elisa.models.parameter.ConstantInterval)
parameters to a model component, the model will be evaluated according to the
following formula:

$$\frac{1}{\prod_i (b_i - a_i)} \int f(E, \vec{\theta}(\vec{p}, \vec{q})) \, \mathrm{d} \vec{p}$$

where $f$ is the model function, $\vec{\theta}$ is the parameter vector of the
model, $\vec{p}$ is the [`ConstantInterval`](#elisa.models.parameter.ConstantInterval)
parameters, $\vec{q}$ is the other parameters, and $a_i$ and $b_i$ are
the intervals given by $\vec{p}$.

In [ ]:
from elisa import ConstantInterval

alpha = ConstantInterval(
    name='alpha',
    interval=[1.0, 3.0],
)
m7 = PowerLaw(alpha=alpha)
m7

In [ ]:
photon_egrid = np.linspace(0.3, 15.0, 1000)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
ne = m7.compile().ne(photon_egrid)
plt.step(egrid_mid, ne)
plt.title(m7.name)
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

#### CompositeParameter

[`CompositeParameter`](#elisa.models.parameter.CompositeParameter) combines
multiple parameters into a single parameter. The value of the composite
parameter is calculated by a user-defined function that takes the values of
the constituent parameters as input. Note that the function must be ``JAX``
compatible.

It is the key for linking parameters across different model components. For
example, we can create a double [`Blackbody`](#elisa.models.add.Blackbody)
model, forcing the temperature of one component to be smaller than that of the
other,

In [ ]:
import jax.numpy as jnp

from elisa import CompositeParameter

# Define the temperature parameter of the first blackbody
kT1 = UniformParameter(
    name='kT',
    default=10.0,
    min=0.1,
    max=150.0,
)

# Define a factor lies in [0.01, 1.0]
f = UniformParameter(
    name='f',
    default=0.2,
    min=0.01,
    max=1.0,
)

# Use CompositeParameter to define the temperature
# parameter of the second Blackbody, so that it is
# always smaller than kT1
kT2 = CompositeParameter(
    params=[f, kT1],
    op=lambda x, y: jnp.multiply(x, y),  # x * y also works
    op_name='{} * {}',
)
m8 = Blackbody(kT=kT1) + Blackbody(kT=kT2)
m8

In [ ]:
photon_egrid = np.linspace(1, 500.0, 1000)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
compiled_model = m8.compile()
vFv = compiled_model.eene(photon_egrid)
bb1 = compiled_model.eene(photon_egrid, params={'Blackbody_2.K': 0.0})
bb2 = compiled_model.eene(photon_egrid, params={'Blackbody.K': 0.0})
plt.step(egrid_mid, vFv, label='total')
plt.step(egrid_mid, bb1, ls=':', label='Blackbody')
plt.step(egrid_mid, bb2, ls=':', label='Blackbody_2')
plt.legend()
plt.title(m8.name)
plt.xlabel('Energy [keV]')
plt.ylabel(r'$\nu F_\nu$ [erg cm$^{-2}$ s$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

We can create a series of power-law models, whose photon indices are evolving
from hard to soft,

In [ ]:
# Create 10 time bins
n = 5
time_grid = np.linspace(0.0, 5.0, n + 1)

# The initial photon index
a0 = UniformParameter(
    name='a0',
    default=0.5,
    min=0.1,
    max=1.0,
)

# The decreasing rate of photon index
rate = UniformParameter(
    name='r',
    default=1.0,
    min=0.5,
    max=2.0,
)

models = []
for i in range(n):
    t = ConstantInterval('t', time_grid[i : i + 2])
    alpha = a0 + t * rate
    models.append(PowerLaw(alpha=alpha))
models[0]

In [ ]:
models[1]

Note the a0 and r parameters are shared among models.

In [ ]:
# Plot with default parameters values
photon_egrid = np.geomspace(0.1, 10.0, 300)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
for i in range(n):
    ne = models[i].compile().ne(photon_egrid)
    label = f'[{time_grid[i]}, {time_grid[i + 1]}] s'
    plt.step(egrid_mid, ne, label=label)
plt.legend()
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

In [ ]:
# Plot with a0=0, r=0.2
params = {'a0': 0, 'r': 1.5}
photon_egrid = np.geomspace(0.1, 10.0, 300)
egrid_mid = 0.5 * (photon_egrid[:-1] + photon_egrid[1:])
for i in range(n):
    ne = models[i].compile().ne(photon_egrid, params=params)
    label = f'[{time_grid[i]}, {time_grid[i + 1]}] s'
    plt.step(egrid_mid, ne, label=label)
plt.legend()
plt.xlabel('Energy [keV]')
plt.ylabel('$N_E$ [ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$]')
plt.xscale('log')
plt.yscale('log')

When generating the `alpha` parameter for power-law model of each time slice,
we set ``alpha = a0 + t * rate``, rather than using the [`CompositeParameter`](#elisa.models.parameter.CompositeParameter). This is because ``ELISA`` provides a set of arithmetic
operators to create [`CompositeParameter`](#elisa.models.parameter.CompositeParameter)
in a more straightforward way.
We can use these operators on two parameters (including composite ones):

- `+`: sum
- `-`: subtraction
- `*`: multiplication
- `/`: division
- `**`: power

For example,

In [ ]:
x = UniformParameter('x', 5, 1, 10)
y = UniformParameter('y', -2, -5, 10)
z = x**y
print(type(z).__name__, z)

### Manipulating Parameters of Components

We can manipulate the parameters of the components after the model is created.

In [ ]:
m9 = PowerLaw()

# We first get the powerlaw component,
# then get K parameter of the component.
# Note that K is a UniformParameter by default,
# with five attributes: default, min, max, log, and fixed.

K = m9.PowerLaw.K
print('K', type(K).__name__, K.default, K.min, K.max, K.log, K.fixed, sep=', ')
m9

In [ ]:
# Modify the K parameter
m9.PowerLaw.K.default = 1.5
m9['PowerLaw'].K.min = 0.1
m9.PowerLaw['K'].max = 5.0
m9['PowerLaw']['K'].log = True
m9

In [ ]:
# Set K to DistParameter with LogNormal distribution
m9.PowerLaw.K = DistParameter(
    name='K',
    dist=dist.LogNormal(10, 1),
    default=10,
)
m9

A more convenient method to create a model with two blackbodies, as shown in the example above, can be as follows:

In [ ]:
f = UniformParameter('f', 0.5, 0.01, 1.0)
m10 = Blackbody() + Blackbody()
m10.Blackbody_2.kT = f * m10.Blackbody.kT
m10

### Conclusion

The parameter interface of ``ELISA`` provides versatile approaches to configuring
the parameters of model components. You can use parameters with default
configuration, or create parameters with custom values, bounds, and priors.
Additionally, linking parameters across various model components is supported.